# Assigment 4

In [76]:
import math
import numpy as np
import scipy.sparse as sp

from random import seed
from random import random

import igl
import meshplot as mp
mp.offline()
from math import sqrt

In [31]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

In [32]:
def align_field(V, F, TT, soft_id, soft_value, llambda):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    
    sqrtl = sqrt(llambda)
    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    
    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=np.complex)
    
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        
        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c * sqrtl
        
        index += 1
    
    assert(b.shape[0] == index)
    
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.H @ A, A.H @ b)
    
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]
    
    return R

In [33]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col, filename = "test")
    p.add_lines(B, B + R * avg)
    
    return p

In [36]:
R = align_field(v, f, tt, cf, c, 1e6)
plot_mesh_field(v, f, R, cf)
print("Interpolated field:")
print(R)
print("Constraints:")
print(c)

Plot saved to file test.html.
Interpolated field:
[[ 0.23591955 -0.22720053  0.39047184]
 [-0.42277552  0.13051146 -0.14899512]
 [ 0.68709967 -0.24579442 -0.29458209]
 ...
 [ 0.44583999 -0.09987587  0.05341988]
 [ 0.3475595  -0.66810854  0.60711693]
 [ 0.40873459 -0.6707803   0.56895469]]
Constraints:
[[-0.86136    0.050168  -0.505512 ]
 [-0.822122   0.13914   -0.552047 ]
 [-0.761541   0.286908  -0.581154 ]
 [-0.527336   0.718255  -0.453902 ]
 [-0.426421   0.812406  -0.397695 ]
 [-0.173632   0.94519   -0.276528 ]
 [-0.12095    0.987557  -0.100515 ]
 [ 0.528064  -0.685298   0.501512 ]
 [ 0.485416  -0.738721   0.467614 ]
 [ 0.499676  -0.705184   0.50303  ]
 [ 0.566063  -0.704697   0.427757 ]
 [ 0.540817  -0.730976   0.416162 ]
 [ 0.562468  -0.68217    0.467198 ]
 [ 0.604154  -0.712722   0.356406 ]
 [ 0.631683  -0.639429   0.438301 ]
 [ 0.818211  -0.414128   0.398783 ]
 [ 0.817361  -0.21155   -0.53588  ]
 [ 0.772265  -0.0814812 -0.630054 ]
 [ 0.708944  -0.0922904 -0.699201 ]
 [ 0.659804  

C:\Users\nicev\AppData\Local\Temp\ipykernel_21028\434597452.py:57: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  b = np.zeros(index + soft_id.shape[0], dtype=np.complex)


In [70]:
# part 2, I tried my best but have no idea what i'm doing really
def reconstruct_scalar(vector_field):
    # need to solve g = G@s
    s = np.zeros((v.shape[0],1)) # should be #v by 1
    G = np.zeros((f.shape[0]*3,1)) # should be 3#f by 1, use this to hold the barycentric 
    print(G.shape)
    print(s.shape)
    grad = igl.grad(v,f)
    print(grad.shape)
    area = igl.doublearea(v,f) # stores the 2A value
    print(R.shape)
    
    # for every face, calculate the gradient function
    for i in range(len(f)):
        # compute the edges of the face
        v1 = v[f[i][0]]
        v2 = v[f[i][1]]
        v3 = v[f[i][2]]
        e1 = v2-v1
        e2 = v3-v1
        # create a perpendicular vector from this using cross product
        e3 = np.cross(e1,e2)
        
        # now calculate w(perp)/2A
        a1 = e3/area[i]
        G[i*3] = a1[0]
        G[(i*3)+1] = a1[1]
        G[(i*3)+2] = a1[2]

    # need to include this gt.T.reshape(gt.size, 1)
    
    
    
reconstruct_scalar(R)

(2268, 1)
(411, 1)
(2268, 411)
(756, 3)


In [89]:
# harmonic

v, f  = igl.read_triangle_mesh("data/camel_head.off")
## Find the open boundary
bnd = igl.boundary_loop(f)

## Map the boundary to a circle, preserving edge proportions
bnd_uv = igl.map_vertices_to_circle(v, bnd)

## Harmonic parametrization for the internal vertices
uv = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)
v_p = np.hstack([uv, np.zeros((uv.shape[0],1))])

ind = np.zeros((v.shape[0]))

for i in range(uv.shape[0]):
    if uv[i][0] > 0:
        ind[i] = 1
    else:
        ind[i] = 2
print(ind)

# I used plot instead of subplot since subplot is broken on my machine, subplot elements aren't added
p = mp.plot(v, f, c =ind, shading={"wireframe": False, "flat": False},filename="harmonic_camel")
p = mp.plot(v_p, f, uv=uv, c=ind, shading={"wireframe": True, "flat": False}, filename = "flat_harmonic_camel")

[1. 1. 1. ... 1. 1. 1.]
Plot saved to file harmonic_camel.html.
Plot saved to file flat_harmonic_camel.html.


In [92]:
# lscm

v, f = igl.read_triangle_mesh("data/camel_head.off")

# Fix two points on the boundary
b = np.array([2, 1])

bnd = igl.boundary_loop(f)
b[0] = bnd[0]
b[1] = bnd[int(bnd.size / 2)]

bc = np.array([[0.0, 0.0], [1.0, 0.0]])

# LSCM parametrization
_, uv = igl.lscm(v, f, b, bc)

ind = np.zeros((v.shape[0]))

for i in range(uv.shape[0]):
    if uv[i][0] > 0:
        ind[i] = 1
    else:
        ind[i] = 2

# I used plot instead of subplot since subplot is broken on my machine, subplot elements aren't added
p = mp.plot(v, f, c=ind, shading={"wireframe": False, "flat": False}, filename="lscm_camel")
p = mp.plot(uv, f, uv=uv, shading={"wireframe": False, "flat": False}, filename="flat_lscm_camel")

Plot saved to file lscm_camel.html.
Plot saved to file flat_lscm_camel.html.


In [88]:
# part 4, can only give the vibe of how I would do it since I didn't complete part 2 so I con't complete the pipeline
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
# use part three to create the lscm
# Fix two points on the boundary
b = np.array([2, 1])

bnd = igl.boundary_loop(f)
b[0] = bnd[0]
b[1] = bnd[int(bnd.size / 2)]

bc = np.array([[0.0, 0.0], [1.0, 0.0]])

# LSCM parametrization
_, uv = igl.lscm(v, f, b, bc)
print(uv.shape)

# use part 1 to create vector field from constraints
vector_field = align_field(v, f, tt, cf, c, 1e6)

# use part 2 to create scalar field from vector field
scalar_field = reconstruct_scalar(vector_field)

# done since reconstruct_scalar is broken
scalar_field = np.zeros((v.shape[0],1))
for i in range(len(scalar_field)):
    scalar_field[i] = 0.5

# replace the v in uv with the scalar_field
for i in range(len(uv)):
    uv[i][1] = scalar_field[i]

# output wrong due to skipping scalar field calculation
p = mp.plot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, filename="edited_parameterization")

(411, 2)
(2268, 1)
(411, 1)
(2268, 411)
(756, 3)
Plot saved to file edited_parameterization.html.


C:\Users\nicev\AppData\Local\Temp\ipykernel_21028\434597452.py:57: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  b = np.zeros(index + soft_id.shape[0], dtype=np.complex)
